In [ ]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
import time

In [ ]:
spark

In [ ]:
#config
#spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)
#ps.set_option("compute.default_index_type", "distributed")

### customer data eda

In [ ]:
customers_pd = pd.read_csv('/content/h_m_dataset/customers.csv')

In [ ]:
customers_pd['FN'] = customers_pd['FN'].astype("object")
customers_pd['Active'] = customers_pd['Active'].astype("object")
customers_pd.info()

print(customers_pd.info())
print(customers_pd.head(5))
print(customers_pd.describe())

customers_ps = ps.from_pandas(customers_pd)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137198 entries, 0 to 137197
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              137198 non-null  int64  
 1   customer_id             137198 non-null  object 
 2   FN                      47980 non-null   object 
 3   Active                  46761 non-null   object 
 4   club_member_status      136591 non-null  object 
 5   fashion_news_frequency  135574 non-null  object 
 6   age                     135576 non-null  float64
 7   postal_code             137198 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 8.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137198 entries, 0 to 137197
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              137198 non-null  int64  
 1   customer_id

In [ ]:
# missing value test
customers_colna_perc = customers_ps.isnull().sum()/len(customers_ps)*100
print(customers_colna_perc)

Unnamed: 0                 0.000000
customer_id                0.000000
FN                        65.028645
Active                    65.917142
club_member_status         0.442426
fashion_news_frequency     1.183691
age                        1.182233
postal_code                0.000000
dtype: float64


In [ ]:
# one-variable distribution(if object than value_counts(dropna=False),float64 or int32 than histogram)
# 1.turn your column type to object or int(float)
# 2.object -> value_counts
# 3.int -> histogram
def one_var_dist(df):
  plot_dict = {}
  for col in df.columns:
    if ((df[col].dtypes in ['object']) & (len(df[col].unique()) < 25)): 
      plot_dict.update({col:df[col].value_counts(dropna=False)})
    elif(df[col].dtypes in ['float64','int32']):
      plot_dict.update({col:df[col].hist(bins=20)})       
    else:
      plot_dict.update({col:'this col has unique values:'+str(len(df[col].unique()))})
  return plot_dict

In [ ]:
plot_dict_customers = one_var_dist(customers_ps)
for key,value in plot_dict_customers.items():
  if str(type(value)) in ['<class \'plotly.graph_objs._figure.Figure\'>']:
      print(key)
      plot_dict_customers[key].show()
  else:
    print(key)
    print(value)
  print('---------------------------------------------------------------')

Unnamed: 0
this col has unique values:137198
---------------------------------------------------------------
customer_id
this col has unique values:137198
---------------------------------------------------------------
FN
NaN    89218
1.0    47980
Name: FN, dtype: int64
---------------------------------------------------------------
Active
NaN    90437
1.0    46761
Name: Active, dtype: int64
---------------------------------------------------------------
club_member_status
ACTIVE        127207
PRE-CREATE      9342
NaN              607
LEFT CLUB         42
Name: club_member_status, dtype: int64
---------------------------------------------------------------
fashion_news_frequency
NONE         87427
Regularly    48052
NaN           1624
Monthly         95
Name: fashion_news_frequency, dtype: int64
---------------------------------------------------------------
age


---------------------------------------------------------------
postal_code
this col has unique values:99517
---------------------------------------------------------------


### article data eda

In [ ]:
articles_pd = pd.read_csv('/content/h_m_dataset/articles.csv')

In [ ]:
articles_pd['article_id'] = articles_pd['article_id'].astype("object")
articles_pd['product_code'] = articles_pd['product_code'].astype("object")
articles_pd['product_type_no'] = articles_pd['product_type_no'].astype("object")
articles_pd['graphical_appearance_no'] = articles_pd['graphical_appearance_no'].astype("object")
articles_pd['colour_group_code'] = articles_pd['colour_group_code'].astype("object")
articles_pd['perceived_colour_value_id'] = articles_pd['perceived_colour_value_id'].astype("object")
articles_pd['perceived_colour_master_id'] = articles_pd['perceived_colour_master_id'].astype("object")
articles_pd['department_no'] = articles_pd['department_no'].astype("object")
articles_pd['index_group_no'] = articles_pd['index_group_no'].astype("object")
articles_pd['section_no'] = articles_pd['section_no'].astype("object")
articles_pd['garment_group_no'] = articles_pd['garment_group_no'].astype("object")
articles_pd.info()

print(articles_pd.info())
print(articles_pd.head(5))
print(articles_pd.describe())

articles_ps = ps.from_pandas(articles_pd)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10554 entries, 0 to 10553
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Unnamed: 0                    10554 non-null  int64 
 1   article_id                    10554 non-null  object
 2   product_code                  10554 non-null  object
 3   prod_name                     10554 non-null  object
 4   product_type_no               10554 non-null  object
 5   product_type_name             10554 non-null  object
 6   product_group_name            10554 non-null  object
 7   graphical_appearance_no       10554 non-null  object
 8   graphical_appearance_name     10554 non-null  object
 9   colour_group_code             10554 non-null  object
 10  colour_group_name             10554 non-null  object
 11  perceived_colour_value_id     10554 non-null  object
 12  perceived_colour_value_name   10554 non-null  object
 13  perceived_colour

In [ ]:
# missing value test
articles_colna_perc = articles_ps.isnull().sum()/len(articles_ps)*100
print(articles_colna_perc)

Unnamed: 0                      0.000000
article_id                      0.000000
product_code                    0.000000
prod_name                       0.000000
product_type_no                 0.000000
product_type_name               0.000000
product_group_name              0.000000
graphical_appearance_no         0.000000
graphical_appearance_name       0.000000
colour_group_code               0.000000
colour_group_name               0.000000
perceived_colour_value_id       0.000000
perceived_colour_value_name     0.000000
perceived_colour_master_id      0.000000
perceived_colour_master_name    0.000000
department_no                   0.000000
department_name                 0.000000
index_code                      0.000000
index_name                      0.000000
index_group_no                  0.000000
index_group_name                0.000000
section_no                      0.000000
section_name                    0.000000
garment_group_no                0.000000
garment_group_na

In [ ]:
plot_dict_articles = one_var_dist(articles_ps)
for key,value in plot_dict_articles.items():
  if str(type(value)) in ['<class \'plotly.graph_objs._figure.Figure\'>']:
      print(key)
      plot_dict_articles[key].show()
  else:
    print(key)
    print(value)
  print('---------------------------------------------------------------')

Unnamed: 0
this col has unique values:10554
---------------------------------------------------------------
article_id
this col has unique values:10554
---------------------------------------------------------------
product_code
this col has unique values:8994
---------------------------------------------------------------
prod_name
this col has unique values:8943
---------------------------------------------------------------
product_type_no
this col has unique values:101
---------------------------------------------------------------
product_type_name
this col has unique values:101
---------------------------------------------------------------
product_group_name
Garment Upper body     4237
Garment Lower body     1961
Garment Full body      1336
Accessories            1178
Shoes                   561
Underwear               521
Swimwear                317
Socks & Tights          213
Nightwear               195
Unknown                  17
Bags                      6
Cosmetic          

### transaction data eda

In [ ]:
transactions_pd = pd.read_csv('/content/h_m_dataset/transactions_train.csv')

In [ ]:
# turn column type 
transactions_pd['article_id'] = transactions_pd['article_id'].astype("object")
transactions_pd['sales_channel_id'] = transactions_pd['sales_channel_id'].astype("object")
transactions_pd.info()

print(transactions_pd.info())
print(transactions_pd.head(5))
print(transactions_pd.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317883 entries, 0 to 317882
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        317883 non-null  int64  
 1   t_dat             317883 non-null  object 
 2   customer_id       317883 non-null  object 
 3   article_id        317883 non-null  object 
 4   price             317883 non-null  float64
 5   sales_channel_id  317883 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 14.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317883 entries, 0 to 317882
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        317883 non-null  int64  
 1   t_dat             317883 non-null  object 
 2   customer_id       317883 non-null  object 
 3   article_id        317883 non-null  object 
 4   price             317883 non-null  float

In [ ]:
transactions_ps = ps.from_pandas(transactions_pd)

In [ ]:
# missing value test
transactions_colna_perc = transactions_ps.isnull().sum()/len(transactions_ps)*100
print(transactions_colna_perc)

Unnamed: 0          0.0
t_dat               0.0
customer_id         0.0
article_id          0.0
price               0.0
sales_channel_id    0.0
dtype: float64


In [ ]:
plot_dict_transactions = one_var_dist(transactions_ps)

for key,value in plot_dict_transactions.items():
  if str(type(value)) in ['<class \'plotly.graph_objs._figure.Figure\'>']:
      print(key)
      plot_dict_transactions[key].show()
  else:
    print(key)
    print(value)
  print('---------------------------------------------------------------')

Unnamed: 0
this col has unique values:317883
---------------------------------------------------------------
t_dat
this col has unique values:734
---------------------------------------------------------------
customer_id
this col has unique values:230222
---------------------------------------------------------------
article_id
this col has unique values:53890
---------------------------------------------------------------
price


---------------------------------------------------------------
sales_channel_id
2    223763
1     94120
Name: sales_channel_id, dtype: int64
---------------------------------------------------------------


''